In [1]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc

In [28]:
######
EX = 1
######


if EX == 1:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2041", user_risk_profile = 0)
    # set planned liabilities
    problem.liabilities.insert("car", "Jan 2026", 25000, 25000*0.65)
    problem.liabilities.insert("university", "Jan 2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "Jan 2037",25000, 25000*0.85)
    # set planned assets 
    problem.assets.insert("ass_0","Jan 2021",30000)
    recurrent_dates = ["Jan 2022", "Jan 2023", "Jan 2024", "Jan 2025", "Jan 2026", "Jan 2027"]
    for i in np.arange(len(recurrent_dates)):
        problem.assets.insert("ass_" + str(i+1),recurrent_dates[i],10000)
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061")
    # set planned liabilities
    recurrent_assets = pd.date_range(start = "Jan 2021", end="Jan 2040", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_assets)):
        problem.assets.insert("ass_" + str(i+1),recurrent_assets[i],1000)
    recurrent_liabilities = pd.date_range(start = "Jan 2041", end="Jan 2060", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_liabilities)):
        problem.liabilities.insert("liab_" + str(i+1),recurrent_liabilities[i],1500,1100)

In [29]:
ALMc.display(problem,bar_width = 6)

In [30]:
problem.check_feasibility()
problem.get_feasibility()

check feasibility ended in 9.56 s with Optimal solution
Feasible problem


In [31]:
ALMc.display(problem,bar_width = 6)

In [32]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve(problem)

BaH_model = ALM.ALMBuyAndHold(problem)
BaH_model.solve(problem)

GoalBased model generated in 8.57 s
Solve ended in 0.98 s with Optimal solution
BuyAndHold model generated in 8.41 s
Solve ended in 0.74 s with Optimal solution


# Display solution

In [33]:
sol = GB_model.solution
sol_bah = BaH_model.solution


In [34]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "ETF")
plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "ETF")
plt.show()

In [35]:
plt = ALMc.AssetSplitDetailsChart(problem, sol, "Goal")
plt.show()
plt = ALMc.AssetSplitDetailsChart(problem, sol_bah, "Goal")
plt.show()

In [36]:
n_scen = None
perc = True
#print([solution[l][n_scen].varValue for l in Lt])
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc)
plt.show()
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc)
plt.show()

In [37]:
P = problem.P
A = problem.assets.set
L = problem.liabilities.set
N = problem.N
T = problem.T
Scenario = problem.Scenario

In [38]:
def show_named_plotly_colours():
    """
    function to display to user the colours to match plotly's named
    css colours.

    Reference:
        #https://community.plotly.com/t/plotly-colours-list/11730/3

    Returns:
        plotly dataframe with cell colour to match named colour name

    """
    s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, black, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
    li=s.split(',')
    li=[l.replace('\n','') for l in li]
    li=[l.replace(' ','') for l in li]

    import pandas as pd
    import plotly.graph_objects as go

    df=pd.DataFrame.from_dict({'colour': li})
    fig = go.Figure(data=[go.Table(
      header=dict(
        values=["Plotly Named CSS colours"],
        line_color='black', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[df.colour],
        line_color=[df.colour], fill_color=[df.colour],
        align='center', font=dict(color='black', size=11)
      ))
    ])

    fig.show()

show_named_plotly_colours()

In [39]:
Q_ln = {}
for l in L:
    Q_ln[l] = np.zeros(shape = (len(N)))
    for n in N:
        Q_ln[l][n]=sol.goal_distr[l][N[n]]
Q_ln["ex_wealth"] = np.zeros(shape = (len(N)))
for n in N:
    Q_ln["ex_wealth"][n]=sol.final_exwealth[N[n]]
df_Q_ln = pd.DataFrame(Q_ln)

In [40]:
df = pd.DataFrame(index = problem.liabilities.set)
conf_tg = pd.DataFrame(problem.liabilities.value_tg, columns = problem.liabilities.set, index = N)
conf_lb = pd.DataFrame(problem.liabilities.value_lb, columns = problem.liabilities.set, index = N)

df["Affordable"] = np.logical_and(df_Q_ln[problem.liabilities.set] < conf_tg, df_Q_ln[problem.liabilities.set] >= conf_lb).mean()
df["Failure"] = (df_Q_ln[problem.liabilities.set] < conf_lb).mean()
df = df.reset_index().rename(columns = {"index":"Goals"})
dfchart = pd.melt(df, id_vars=["Goals"], value_vars = ["Affordable", "Failure"], var_name = "Shortfall Cathegory", value_name = "Shortfall Probabilities")
plot = px.bar(dfchart, x="Goals", y = "Shortfall Probabilities", color = "Shortfall Cathegory",color_discrete_sequence=['royalblue', "crimson"])
plot = ALMc.standardized_chart(plot, perc = True)
plot.show()

In [61]:
exp_wealth = df_Q_ln.mean()
df = pd.DataFrame(index = problem.liabilities.set)
print(df)

df["goal"] = pd.Series(problem.liabilities.value_tg, index = problem.liabilities.set)
df["avg"] = np.round(df_Q_ln[problem.liabilities.set][(df_Q_ln[problem.liabilities.set] < conf_tg[problem.liabilities.set])].mean(),0)
df["avg_shortfall"] = df["goal"] - df["avg"]
df["lower_bound"] = pd.Series(problem.liabilities.value_lb, index = problem.liabilities.set)
df["worst"] = np.round(df_Q_ln[problem.liabilities.set][(df_Q_ln[problem.liabilities.set] <= np.quantile(df_Q_ln[problem.liabilities.set],0.05,axis = 0))].mean(),0)
print(df)

Empty DataFrame
Columns: []
Index: [car, university, hawaii]
             goal      avg  avg_shortfall  lower_bound    worst
car         25000  20768.0         4232.0      16250.0  16285.0
university  50000  48901.0         1099.0      47500.0  47540.0
hawaii      25000  22985.0         2015.0      21250.0  21358.0


In [84]:
fig = go.Figure(
    data = [
            go.Scatter(x = df.index, y=df["worst"], mode = "markers", name ="worst 5%", marker_color = "red"),
            go.Bar(x = df.index, y=df["goal"], marker_color = "gold", name = "Goal"),
            go.Bar(x = df.index, y=df["avg"], marker_color = "limegreen", name = "Average Value"),
            ],
    layout = go.Layout(barmode = "overlay")
        )
fig = ALMc.standardized_chart(fig)
fig.show()